In [ ]:
import dlt, requests
import pandas as pd

In [ ]:
URL = "https://jaffle-shop.scalevector.ai/api/v1/customers"

req = requests.get(URL)

df = pd.json_normalize(req.json())

df.head()

In [ ]:
# 1. dlt를 사용하여 S3의 CSV 파일을 데이터 소스로 읽어오는 코드 예시입니다.

import dlt
import pandas as pd
from dlt.sources.filesystem import filesystem

# dlt의 s3 파일 커넥터를 사용하여 S3의 CSV 파일을 읽어옵니다.
# dlt.sources.filesystem.filesystem_source를 사용합니다.

filesystem_source = filesystem(
  bucket_url="s3://lakeformation-demo-hyunsoo/dlt_dataset/",
  file_glob="*.csv"
)

# 데이터를 DataFrame으로 변환
emp_df = pd.DataFrame(list(filesystem_source))

# 데이터 확인
emp_df.head()


In [ ]:
# S3에서 읽은 emp_df(DataFrame)를 PostgreSQL에 저장하는 dlt 파이프라인 생성
emp_pg_pipeline = dlt.pipeline(
    pipeline_name="emp_data_pg_pipeline3",
    destination="postgres",
    dataset_name="emp_data_pg"
)

# emp_df를 "employees" 테이블로 적재
emp_pg_load_info = emp_pg_pipeline.run(
    data=filesystem_source,
    table_name="employees3"
)

print("S3 CSV 데이터를 PostgreSQL에 저장한 결과:", emp_pg_load_info)


In [ ]:
pipeline = dlt.pipeline(
    pipeline_name="customer_data_pipeline",
    destination="filesystem",
    dataset_name="customer_data"
)

load_info = pipeline.run(
    data=df, 
    table_name="customers"
    )

print("파이프라인 실행 결과:", load_info)

In [ ]:
pd.read_parquet('./api_data_dir/customer_data/customers/')

In [ ]:
import dlt

# PostgreSQL로 데이터를 저장하는 dlt 파이프라인 생성
# 동일 코드라도 데이터가 다르면 pipeline_name을 다르게 설정해야 함
pg_pipeline = dlt.pipeline(
    pipeline_name="customer_data_pg_pipeline2",
    destination="postgres",
    dataset_name="customer_data_pg"
)

# 데이터 적재 실행
pg_load_info = pg_pipeline.run(
    data=df,
    table_name="stores"
)

print("PostgreSQL 파이프라인 실행 결과:", pg_load_info)



In [ ]:
import dlt

# AWS Athena로 데이터를 저장하는 dlt 파이프라인 생성
athena_pipeline = dlt.pipeline(
    pipeline_name="customer_data_athena_pipeline",
    destination="athena",
    dataset_name="customer_data_athena"
)

# 데이터 적재 실행
athena_load_info = athena_pipeline.run(
    data=df,
    table_name="customers"
)

print("Athena 파이프라인 실행 결과:", athena_load_info)

In [ ]:


# Athena에서 S3 버킷에 접근 권한이 없어서 발생하는 에러입니다.
# Lake Formation 또는 S3 버킷의 권한 설정을 확인해야 합니다.
# 아래는 문제 해결을 위한 안내 코드와 설명입니다.

print("""
[문제 원인]
- Athena가 S3 버킷(s3://personal-golight-image-bucket/dataset/customer_data_athena/_dlt_pipeline_state)에 접근할 권한이 없습니다.
- Lake Formation 또는 S3 버킷의 권한 정책에서 Athena(및 Glue, Data Catalog) 역할/사용자에게 충분한 권한이 부여되어야 합니다.

[해결 방법]
1. AWS 콘솔에서 S3 버킷(s3://personal-golight-image-bucket)에 대해 다음 권한을 부여하세요.
   - s3:GetObject, s3:PutObject, s3:DeleteObject, s3:ListBucket 등
   - 권한을 부여할 주체: Athena, Glue, Data Catalog에서 사용하는 IAM Role 또는 User

2. Lake Formation을 사용하는 경우:
   - Lake Formation 콘솔에서 해당 S3 경로에 대해 Athena, Glue, Data Catalog에 '데이터 위치 권한'을 부여하세요.
   - '데이터 위치' 등록 및 '데이터 위치 권한' 부여 필요

3. 권한이 정상적으로 부여된 후 파이프라인을 다시 실행하세요.

[예시: S3 버킷 정책]
{
  "Effect": "Allow",
  "Principal": {
    "Service": [
      "athena.amazonaws.com",
      "glue.amazonaws.com"
    ],
    "AWS": "arn:aws:iam::<YOUR_ACCOUNT_ID>:role/<YOUR_ATHENA_GLUE_ROLE>"
  },
  "Action": [
    "s3:GetObject",
    "s3:PutObject",
    "s3:DeleteObject",
    "s3:ListBucket"
  ],
  "Resource": [
    "arn:aws:s3:::personal-golight-image-bucket/dataset/*",
    "arn:aws:s3:::personal-golight-image-bucket/dataset"
  ]
}

[참고]
- 권한 변경 후에도 문제가 지속되면, Lake Formation에서 '데이터 위치 권한'과 '테이블 권한'을 모두 확인하세요.
- 자세한 내용은 AWS 공식 문서(https://docs.aws.amazon.com/ko_kr/athena/latest/ug/lake-formation.html) 참고

""")


In [ ]:
import dlt, requests
import pandas as pd


def get_customer_data(url: str) -> pd.DataFrame:
    """API로부터 고객 데이터를 가져오는 함수"""
    response = requests.get(url)
    df = pd.json_normalize(response.json())
    return df
    
def load_to_s3(data: pd.DataFrame, **kwargs):
    """데이터를 S3에 적재하는 함수"""
    pipeline = dlt.pipeline(
        pipeline_name=kwargs["pipeline_name"],
        destination="filesystem", 
        dataset_name=kwargs["dataset_name"]
    )
    return pipeline

In [ ]:
API_URL = "https://jaffle-shop.scalevector.ai/api/v1/customers"

# 데이터 가져오기
customer_df = get_customer_data(API_URL)

# S3에 데이터 적재
pipeline = load_to_s3(
    data=customer_df,
    pipeline_name="customer_data_pipeline",
    dataset_name="customer_data"
    )

In [ ]:
pipeline.run(
    data=customer_df, 
    table_name="customers",
    write_disposition="append"
    )

In [ ]:
import dlt, requests
import pandas as pd
from dlt.sources.filesystem import filesystem


def get_customer_data(url: str) -> pd.DataFrame:
    """API로부터 고객 데이터를 가져오는 함수"""
    response = requests.get(url)
    df = pd.json_normalize(response.json())
    return df

def load_to_s3(data: pd.DataFrame, **kwargs):
    """데이터를 S3에 적재하는 함수"""
    pipeline = dlt.pipeline(
        pipeline_name=kwargs["pipeline_name"],
        destination="filesystem", 
        dataset_name=kwargs["dataset_name"]
    )
    return pipeline

if __name__ == "__main__":
    # API URL 설정
    API_URL = "https://jaffle-shop.scalevector.ai/api/v1/customers"
    
    # 데이터 가져오기
    customer_df = get_customer_data(API_URL)
    
    # S3에 데이터 적재
    pipeline = load_to_s3(
        data=customer_df,
        pipeline_name="customer_data_pipeline",
        dataset_name="customer_data"
        )
    
    pipeline.run(data=customer_df, table_name="customers")
    
    print("파이프라인 실행 결과:", pipeline)




In [1]:
import dlt, requests
import pandas as pd

def get_customer_data(url: str) -> pd.DataFrame:
    """API로부터 고객 데이터를 가져오는 함수"""
    response = requests.get(url)
    df = pd.json_normalize(response.json())
    return df

def load_to_bigquery(data: pd.DataFrame, **kwargs):
    """데이터를 BigQuery에 적재하는 함수"""
    pipeline = dlt.pipeline(
        pipeline_name=kwargs["pipeline_name"],
        destination="bigquery", 
        dataset_name=kwargs["dataset_name"]
    )
    return pipeline

# API URL 설정
API_URL = "https://jaffle-shop.scalevector.ai/api/v1/customers"

# 데이터 가져오기
customer_df = get_customer_data(API_URL)

# BigQuery에 데이터 적재
pipeline = load_to_bigquery(
    data=customer_df,
    pipeline_name="api_data_pipeline_bq",
    dataset_name="bq_dlt"
)

result = pipeline.run(data=customer_df, table_name="customers")

print("BigQuery 파이프라인 실행 결과:", result)

d:\코드잇\codeit_DA_lecture\lecture\Lib\site-packages\airflow\__init__.py:45 RuntimeWarning: Airflow currently can be run on POSIX-compliant Operating Systems. For development, it is regularly tested on fairly modern Linux Distros and recent versions of macOS. On Windows you can run it via WSL2 (Windows Subsystem for Linux 2) or via Linux Containers. The work to add Windows support is tracked via https://github.com/apache/airflow/issues/10388, but it is not a high priority.

d:\코드잇\codeit_DA_lecture\lecture\Lib\site-packages\airflow\__init__.py:45 RuntimeWarning: Airflow currently can be run on POSIX-compliant Operating Systems. For development, it is regularly tested on fairly modern Linux Distros and recent versions of macOS. On Windows you can run it via WSL2 (Windows Subsystem for Linux 2) or via Linux Containers. The work to add Windows support is tracked via https://github.com/apache/airflow/issues/10388, but it is not a high priority.

BigQuery 파이프라인 실행 결과: Pipeline api_data_pipeline_bq load step completed in 13.44 seconds
1 load package(s) were loaded to destination bigquery and into dataset bq_dlt
The bigquery destination used de-study-sa@codeit-hyunsoo.iam.gserviceaccount.com@codeit-hyunsoo location to store data
Load package 1752074983.7651272 is LOADED and contains no failed jobs
